In [29]:
import pandas as pd
import json
import streamlit as st
import wget
import sqlite3

### captura dos dados

In [30]:
countries = pd.read_json('https://gco.iarc.fr/causes/obesity/data/countries.json')
cancers = pd.read_json('https://gco.iarc.fr/causes/obesity/data/cancers.json')

In [31]:
#attributable = wget.download('https://gco.iarc.fr/causes/obesity/data/attributable.json', out='../data/attributable.json')
#preventable = wget.download('https://gco.iarc.fr/causes/obesity/data/preventable.json', out='../data/preventable.json')

In [32]:
with open('../data/attributable.json') as json_data:
    data = json.load(json_data)
    attributable = pd.json_normalize(data['both'])

In [33]:
with open('../data/preventable.json') as json_data:
    data = json.load(json_data)
    preventable = pd.json_normalize(data['both'])

In [34]:
['Argentina', 'Bahamas', 'Barbados', 'Belize', 'Bolivia', 'Brazil',
       'Chile', 'Colombia', 'Costa Rica', 'Cuba', 'Dominican Republic',
       'Ecuador', 'El Salvador', 'Guatemala', 'Guyana', 'Haiti',
       'Honduras', 'Jamaica', 'Mexico', 'Nicaragua', 'Panama', 'Paraguay',
       'Peru', 'Puerto Rico', 'Suriname', 'Trinidad and Tobago',
       'Uruguay', 'Venezuela (Bolivarian Republic of)']

['Argentina',
 'Bahamas',
 'Barbados',
 'Belize',
 'Bolivia',
 'Brazil',
 'Chile',
 'Colombia',
 'Costa Rica',
 'Cuba',
 'Dominican Republic',
 'Ecuador',
 'El Salvador',
 'Guatemala',
 'Guyana',
 'Haiti',
 'Honduras',
 'Jamaica',
 'Mexico',
 'Nicaragua',
 'Panama',
 'Paraguay',
 'Peru',
 'Puerto Rico',
 'Suriname',
 'Trinidad and Tobago',
 'Uruguay',
 'Venezuela (Bolivarian Republic of)']

In [35]:
attributable.head()

,attr_cases,total_m,total_f,paf_all,sum_attr_cases,inc_m,inc_f,attr_cases_m,attr_cases_f,paf_obs_m,paf_obs_f,paf_obs_both,country.id,country.name,country.WHO_REGION,country.WHO_NAME,region.id,region.name,cancer.id,cancer.name
0,2.503543,3464,5653,0.000275,2.503543,12.196691,6.848275,1.240256,1.263288,0.101688,0.184468,0.131454,1,Angola,AFRO,WHO Africa region,1,sub-Saharan Africa,1,Oesophageal adenocarcinoma
1,5.298474,3464,5653,0.000307,2.794930,56.000000,44.000000,1.316683,1.478247,0.023512,0.033597,0.027949,1,Angola,AFRO,WHO Africa region,1,sub-Saharan Africa,2,Pancreas
2,10.490706,3464,5653,0.000570,5.192232,31.000000,36.000000,1.323300,3.868932,0.042687,0.107470,0.077496,1,Angola,AFRO,WHO Africa region,1,sub-Saharan Africa,3,Kidney
3,19.403432,3464,5653,0.000978,8.912726,123.965108,140.032852,4.454331,4.458396,0.035932,0.031838,0.033761,1,Angola,AFRO,WHO Africa region,1,sub-Saharan Africa,4,Colon
4,22.269271,3464,5653,0.000314,2.865839,80.863397,92.183194,1.414914,1.450925,0.017498,0.015740,0.016561,1,Angola,AFRO,WHO Africa region,1,sub-Saharan Africa,5,Rectum


In [36]:
attributable = attributable[['total_m', 'total_f', 'country.name', 'region.name', 'country.WHO_NAME', 'cancer.name', 'paf_obs_m', 'paf_obs_f', 'paf_obs_both']]
attributable = attributable.rename(columns={"country.name":"country", "country.WHO_NAME":"continent", "cancer.name":"cancer", "region.name":"region"})
attributable['continent'] = attributable['continent'].apply(lambda x: x.replace("WHO ",""))

In [37]:
preventable = preventable[['total_m', 'total_f', 'country.name', "region.name", 'country.WHO_NAME', 'cancer.name', 'paf_obs_m', 'paf_obs_f', 'paf_obs_both']]
preventable = preventable.rename(columns={"country.name":"country", "country.WHO_NAME":"continent", "cancer.name":"cancer", "region.name":"region"})
preventable['continent'] = preventable['continent'].apply(lambda x: x.replace("WHO ",""))

In [38]:
con = sqlite3.connect("../data/tutorial.db")
join = pd.read_sql('select a.country from attributable a left join preventable p on a.country = p.country', con=con)

In [39]:
attributable.head()

,total_m,total_f,country,region,continent,cancer,paf_obs_m,paf_obs_f,paf_obs_both
0,3464,5653,Angola,sub-Saharan Africa,Africa region,Oesophageal adenocarcinoma,0.101688,0.184468,0.131454
1,3464,5653,Angola,sub-Saharan Africa,Africa region,Pancreas,0.023512,0.033597,0.027949
2,3464,5653,Angola,sub-Saharan Africa,Africa region,Kidney,0.042687,0.107470,0.077496
3,3464,5653,Angola,sub-Saharan Africa,Africa region,Colon,0.035932,0.031838,0.033761
4,3464,5653,Angola,sub-Saharan Africa,Africa region,Rectum,0.017498,0.015740,0.016561


In [40]:
con = sqlite3.connect("../data/tutorial.db")

In [41]:
attributable.to_sql('attributable', con=con, if_exists='replace')
preventable.to_sql('preventable', con=con, if_exists='replace')

1584